<a href="https://colab.research.google.com/github/touseefyounas/Sankey-Stocks/blob/main/YahooFinanceAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import requests to call the API; import pandas to convert response list/dict into Dataframes; import numpy to use array functions
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go
#Stock ticker the statement is needed for and to be insterted in the URL
stock='IBM'

#url for the API requests call
url = f"https://yahoo-finance15.p.rapidapi.com/api/yahoo/qu/quote/{stock}/income-statement"


headers = {
	"X-RapidAPI-Key": "8cd9a70cc4mshde5422922530be0p16941bjsn4d9b78c54e6d",
	"X-RapidAPI-Host": "yahoo-finance15.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

#store the response json file in to the data
data=response.json()

#assign the annual income statement into the data value
data=data['incomeStatementHistory']['incomeStatementHistory']


fmt_values = []
for statement in data:
    statement_values = {}
    for key, value in statement.items():
        if isinstance(value, dict) and 'fmt' in value:
            statement_values[key] = value['fmt']
        elif isinstance(value, list):
            # Handle empty lists by setting them to None
            statement_values[key] = np.nan
    fmt_values.append(statement_values)

# Resulting list of income statement values
fmt_df=pd.DataFrame(fmt_values)

raw_values = []
for statement in data:
    statement_values = {}
    for key, value in statement.items():
        if isinstance(value, dict) and 'raw' in value:
            statement_values[key] = value['raw']
        elif isinstance(value, list):
            # Handle empty lists by setting them to None
            statement_values[key] = np.nan
    raw_values.append(statement_values)

raw_df=pd.DataFrame(raw_values)
raw_df

,endDate,totalRevenue,costOfRevenue,grossProfit,researchDevelopment,sellingGeneralAdministrative,nonRecurring,otherOperatingExpenses,totalOperatingExpenses,operatingIncome,...,incomeBeforeTax,incomeTaxExpense,minorityInterest,netIncomeFromContinuingOps,discontinuedOperations,extraordinaryItems,effectOfAccountingCharges,otherItems,netIncome,netIncomeApplicableToCommonShares
0,1672444800,60530000000,27842000000,32688000000,6567000000,19196000000,NaN,-642000000,52963000000,7567000000,...,1156000000,-626000000,77000000,1782000000,-143000000,NaN,NaN,NaN,1639000000,1639000000
1,1640908800,57351000000,25865000000,31486000000,6488000000,19803000000,NaN,-577000000,51579000000,5772000000,...,4837000000,124000000,95000000,4713000000,1030000000,NaN,NaN,NaN,5743000000,5743000000
2,1609372800,55179000000,24314000000,30865000000,6262000000,20561000000,NaN,-620000000,50517000000,4662000000,...,2572000000,-1360000000,129000000,3932000000,1658000000,NaN,NaN,NaN,5590000000,5590000000
3,1577750400,57714000000,26180000000,31534000000,5910000000,18724000000,NaN,-639000000,50175000000,7539000000,...,7206000000,60000000,144000000,7146000000,2285000000,NaN,NaN,NaN,9431000000,9431000000


In [13]:
sources=[]
targets=[]
values=[]
link_colors=[]

if raw_df['otherOperatingExpenses'].iloc[0] < 0 or pd.isna(raw_df['otherOperatingExpenses'].iloc[0]):

  if raw_df['incomeTaxExpense'].iloc[0] < 0:
     if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,6,6,7,9,10,10]
          targets = [1,2,6,3,4,7,8,10,10,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#D58A87'] # Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,6,6,7,9,10,11]
          targets = [1,2,6,3,4,7,8,10,10,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'
     if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
        sources = [0,0,1,1,1,6,6,7,9]
        targets = [1,2,6,3,4,7,8,12,12]
        values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                  raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                  abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncome']]
        link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'

  if raw_df['incomeTaxExpense'].iloc[0] > 0:
      if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,6,6,7,7,10,10]
          targets = [1,2,6,3,4,7,8,10,9,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,6,6,7,7,10,11]
          targets = [1,2,6,3,4,7,8,10,9,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          link_colors=[['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E']] #Colors Green:'#A4CC9E',Red: '#D58A87'

      if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
          sources = [0,0,1,1,1,6,6,7,7]
          targets = [1,2,6,3,4,7,8,12,9]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncome']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors - Green:'#A4CC9E',Red: '#D58A87'

if raw_df['otherOperatingExpenses'].iloc[0] > 0 and pd.notna(raw_df['otherOperatingExpenses'].iloc[0]):
  if raw_df['incomeTaxExpense'].iloc[0] < 0:
     if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,1,6,6,7,9,10,10]
          targets = [1,2,6,3,4,5,7,8,10,10,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#D58A87'] # Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,1,6,6,7,9,10,11]
          targets = [1,2,6,3,4,5,7,8,10,10,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],raw_df.loc[0,'researchDevelopment'],
                    raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'
     if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
        sources = [0,0,1,1,1,1,6,6,7,9]
        targets = [1,2,6,3,4,5,7,8,12,12]
        values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                  raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],
                  abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),abs(raw_df.loc[0,'incomeTaxExpense']),raw_df.loc[0,'netIncome']]
        link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E'] #Colors Green:'#A4CC9E',Red: '#D58A87'

  if raw_df['incomeTaxExpense'].iloc[0] > 0:
      if pd.notna(raw_df['discontinuedOperations'].iloc[0]):
        if raw_df['discontinuedOperations'].iloc[0]<0:
          sources = [0,0,1,1,1,1,6,6,7,7,10,10]
          targets = [1,2,6,3,4,5,7,8,10,9,12,11]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],raw_df.loc[0,'researchDevelopment'],
                    raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],abs(raw_df.loc[0,'discontinuedOperations'])]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors Green:'#A4CC9E',Red: '#D58A87'
        if raw_df['discontinuedOperations'].iloc[0]>0:
          sources = [0,0,1,1,1,1,6,6,7,7,10,11]
          targets = [1,2,6,3,4,5,7,8,10,9,12,12]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],raw_df.loc[0,'researchDevelopment'],
                    raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),
                    raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncomeFromContinuingOps'],raw_df.loc[0,'netIncome'],raw_df.loc[0,'discontinuedOperations']]
          ink_colors=[['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#A4CC9E']] #Colors Green:'#A4CC9E',Red: '#D58A87'
      if pd.isna(raw_df['discontinuedOperations'].iloc[0]):
          sources = [0,0,1,1,1,1,6,6,7,7]
          targets = [1,2,6,3,4,5,7,8,12,9]
          values = [raw_df.loc[0,'totalRevenue'],raw_df.loc[0,'grossProfit'],raw_df.loc[0,'costOfRevenue'],raw_df.loc[0,'sellingGeneralAdministrative'],
                    raw_df.loc[0,'researchDevelopment'],raw_df.loc[0,'otherOperatingExpenses'],raw_df.loc[0,'operatingIncome'],raw_df.loc[0,'incomeBeforeTax'],
                    abs(raw_df.loc[0,'totalOtherIncomeExpenseNet']),raw_df.loc[0,'incomeTaxExpense'],raw_df.loc[0,'netIncome']]
          link_colors=['#A4CC9E','#D58A87','#A4CC9E','#D58A87','#D58A87','#D58A87','#A4CC9E','#D58A87','#A4CC9E','#D58A87'] #Colors - Green:'#A4CC9E',Red: '#D58A87'

In [ ]:
latest_raw=raw_df.loc[0] #Over here you can add a list of drop down options with an if statement to iterate for the selected year e.g. 2022= [0], 2021=[1] etc.
latest_raw=pd.DataFrame(latest_raw)
latest_raw

,0
endDate,1.672445e+09
totalRevenue,6.053000e+10
costOfRevenue,2.784200e+10
grossProfit,3.268800e+10
researchDevelopment,6.567000e+09
sellingGeneralAdministrative,1.919600e+10
nonRecurring,NaN
otherOperatingExpenses,-6.420000e+08
totalOperatingExpenses,5.296300e+10
operatingIncome,7.567000e+09


In [ ]:
fmt_df

,endDate,totalRevenue,costOfRevenue,grossProfit,researchDevelopment,sellingGeneralAdministrative,nonRecurring,otherOperatingExpenses,totalOperatingExpenses,operatingIncome,...,incomeBeforeTax,incomeTaxExpense,minorityInterest,netIncomeFromContinuingOps,discontinuedOperations,extraordinaryItems,effectOfAccountingCharges,otherItems,netIncome,netIncomeApplicableToCommonShares
0,2022-12-31,60.53B,27.84B,32.69B,6.57B,19.2B,NaN,-642M,52.96B,7.57B,...,1.16B,-626M,77M,1.78B,-143M,NaN,NaN,NaN,1.64B,1.64B
1,2021-12-31,57.35B,25.86B,31.49B,6.49B,19.8B,NaN,-577M,51.58B,5.77B,...,4.84B,124M,95M,4.71B,1.03B,NaN,NaN,NaN,5.74B,5.74B
2,2020-12-31,55.18B,24.31B,30.86B,6.26B,20.56B,NaN,-620M,50.52B,4.66B,...,2.57B,-1.36B,129M,3.93B,1.66B,NaN,NaN,NaN,5.59B,5.59B
3,2019-12-31,57.71B,26.18B,31.53B,5.91B,18.72B,NaN,-639M,50.17B,7.54B,...,7.21B,60M,144M,7.15B,2.29B,NaN,NaN,NaN,9.43B,9.43B


In [14]:
fig = go.Figure(data=[go.Sankey(
    arrangement="freeform",
    node = dict(
      pad = 30,
      thickness = 20,
      line = dict(color = "white", width =0),
      label = [f"Total Revenue ${fmt_df.at[0,'totalRevenue']}",                                  #0
              f"Gross Profit ${fmt_df.at[0,'grossProfit']}",                                     #1
              f"Cost of Revenue -${fmt_df.at[0,'costOfRevenue']}",                               #2
              f"SG&A -${fmt_df.at[0,'sellingGeneralAdministrative']}",                           #3
              f"R&D -${fmt_df.at[0,'researchDevelopment']}",                                     #4
              f"Other -${fmt_df.at[0,'otherOperatingExpenses']}",                                #5
              f"Operating Income ${fmt_df.at[0,'operatingIncome']}",                             #6
              f"Income Before Tax ${fmt_df.at[0,'incomeBeforeTax']}",                            #7
              f"Other Income Expenses ${fmt_df.at[0,'totalOtherIncomeExpenseNet']}",             #8
              f"Income Tax ${fmt_df.at[0,'incomeTaxExpense']}",                                  #9
              f"Net Income - Cont Ops ${fmt_df.at[0,'netIncomeFromContinuingOps']}",             #10
              f"Discontinued Operations ${fmt_df.at[0,'discontinuedOperations']}",               #11
              f"Net Income ${fmt_df.at[0,'netIncome']}"                                          #12
                ],
      color = ['#519E3F','#519E3F','#BC271B','#BC271B','#BC271B','#BC271B','#519E3F','#519E3F','#519E3F','#519E3F','#519E3F','#519E3F','#519E3F']
    ),
    textfont=dict(family='sans serif',size=20,
                  color='rgba(0,0,0,1)'),
    link = dict(
      source = sources,
      target = targets,
      value = values,
      color = link_colors
))])

# add title to the diagram
fig.update_layout(title={'text':'IBM 2022  Income Statement','y':0.95,'x':0.5,'xanchor':'center','yanchor':'top','font_size':30},yaxis={'categoryorder':'total ascending'},paper_bgcolor='rgb(248,248,255)',
    plot_bgcolor='rgb(248,248,255)',)


fig.show()

In [17]:
income_year=fmt_df
income_year['year'] = fmt_df['endDate'].str[:4]
income_year

,endDate,totalRevenue,costOfRevenue,grossProfit,researchDevelopment,sellingGeneralAdministrative,nonRecurring,otherOperatingExpenses,totalOperatingExpenses,operatingIncome,...,incomeTaxExpense,minorityInterest,netIncomeFromContinuingOps,discontinuedOperations,extraordinaryItems,effectOfAccountingCharges,otherItems,netIncome,netIncomeApplicableToCommonShares,year
0,2022-12-31,60.53B,27.84B,32.69B,6.57B,19.2B,NaN,-642M,52.96B,7.57B,...,-626M,77M,1.78B,-143M,NaN,NaN,NaN,1.64B,1.64B,2022
1,2021-12-31,57.35B,25.86B,31.49B,6.49B,19.8B,NaN,-577M,51.58B,5.77B,...,124M,95M,4.71B,1.03B,NaN,NaN,NaN,5.74B,5.74B,2021
2,2020-12-31,55.18B,24.31B,30.86B,6.26B,20.56B,NaN,-620M,50.52B,4.66B,...,-1.36B,129M,3.93B,1.66B,NaN,NaN,NaN,5.59B,5.59B,2020
3,2019-12-31,57.71B,26.18B,31.53B,5.91B,18.72B,NaN,-639M,50.17B,7.54B,...,60M,144M,7.15B,2.29B,NaN,NaN,NaN,9.43B,9.43B,2019
